<a href="https://colab.research.google.com/github/ssnirgudkar/UNet/blob/main/pspnet_pytorch_datasetpaper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Encoder - se_resnext50_32x4d, Decoder - FPN, 100 epochs. FPN = next generation pspnet 

https://github.com/qubvel/segmentation_models.pytorch/blob/master/examples/cars%20segmentation%20(camvid).ipynb

** our masked image is gray scale with 1 channel (height * width*channel=1). and the pixel values are equal to the class ids. Each pixle value is only 1 value and not (a*b*c). The CV2.imread, with a flag od 0 that reads it as a gray scalae does not work for our images although it works for the other dataset

## image size: (256, 320) #it's height and width

## Use GPU so that CUDA is available

1. train - 2570, val - 740, test - 370 , Epochs - 100 
2. batchsize - 10 for train, 5 for validation. 
3. train running with 4 worker threads, validation with 1.
4. i am not using augmentation, and no existing encoder weights.
5. Training time -   hrs. Testing time -  

#To Do - 
1. Right now dice loss and iau for accuracy. check if we need to select any other metrics 
2. We are not plotting the losses / accuracy. need to check what to use to get the plots. should that be consistent across all the models? in that case should we use tensorboard?

In [ ]:
!nvidia-smi

Fri Oct 29 09:50:33 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.29.05    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [ ]:
# Install required libs
!pip install -U segmentation-models-pytorch albumentations --user

     |████████████████████████████████| 87 kB 4.5 MB/s 
     |████████████████████████████████| 102 kB 49.4 MB/s 
     |████████████████████████████████| 58 kB 8.5 MB/s 
     |████████████████████████████████| 376 kB 56.2 MB/s 
     |████████████████████████████████| 47.6 MB 67 kB/s 
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.6.3-py3-none-any.whl size=12421 sha256=7928d40e787abb454386c09ff47d44f17e03b8fa2953c0fe5eb99b179dd0de9f
  Stored in directory: /root/.cache/pip/wheels/90/6b/0c/f0ad36d00310e65390b0d4c9218ae6250ac579c92540c9097a
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60965 sha256=fd384a5f8191b066c016a19a7371fbaebfe2eb266856652604f3ec65de4e3fc4
  Stored in directory: /root/.cache/pip/wheels/ed/27/e8/9543d42de2740d3544db96aefef63bda3f2c1761b3334f4873
Successfully built efficientnet-pytorch pretrainedmodels


In [ ]:
!pip uninstall -y segmentation-models-pytorch

Found existing installation: segmentation-models-pytorch 0.2.0
Uninstalling segmentation-models-pytorch-0.2.0:
  Successfully uninstalled segmentation-models-pytorch-0.2.0


In [ ]:
!git clone https://github.com/Cadene/pretrained-models.pytorch.git
!git clone https://github.com/qubvel/segmentation_models.pytorch
!git clone https://github.com/alexgkendall/SegNet-Tutorial

Cloning into 'pretrained-models.pytorch'...
remote: Enumerating objects: 803, done.
remote: Total 803 (delta 0), reused 0 (delta 0), pack-reused 803
Receiving objects: 100% (803/803), 522.66 KiB | 11.88 MiB/s, done.
Resolving deltas: 100% (491/491), done.
Cloning into 'segmentation_models.pytorch'...
remote: Enumerating objects: 1517, done.
remote: Counting objects: 100% (240/240), done.
remote: Compressing objects: 100% (153/153), done.
remote: Total 1517 (delta 142), reused 137 (delta 87), pack-reused 1277
Receiving objects: 100% (1517/1517), 4.90 MiB | 18.17 MiB/s, done.
Resolving deltas: 100% (825/825), done.
Cloning into 'SegNet-Tutorial'...
remote: Enumerating objects: 2785, done.
remote: Total 2785 (delta 0), reused 0 (delta 0), pack-reused 2785
Receiving objects: 100% (2785/2785), 340.84 MiB | 28.06 MiB/s, done.
Resolving deltas: 100% (81/81), done.


In [ ]:
#!pip install -U segmentation-models-pytorch albumentations --user
#!pip uninstall -y segmentation-models-pytorch
#!pip install segmentation_models_pytorch

!pip install git+https://github.com/IvyGongoogle/pretrained-models.pytorch
!pip install git+https://github.com/lukemelas/EfficientNet-PyTorch


  Cloning https://github.com/IvyGongoogle/pretrained-models.pytorch to /tmp/pip-req-build-r3eq9wxp
  Running command git clone -q https://github.com/IvyGongoogle/pretrained-models.pytorch /tmp/pip-req-build-r3eq9wxp
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.0-py3-none-any.whl size=60553 sha256=4e8075b8509859672137b5fa959b7d73f20f8b0bc7c8e72e6af94e2ee1a4a04c
  Stored in directory: /tmp/pip-ephem-wheel-cache-c81ax9j9/wheels/a8/51/08/bb41925a23e2225b4ca505b4e1709ce700613282945986a4b0
Successfully built pretrainedmodels
  Attempting uninstall: pretrainedmodels
    Found existing installation: pretrainedmodels 0.7.4
    Uninstalling pretrainedmodels-0.7.4:
      Successfully uninstalled pretrainedmodels-0.7.4
  Cloning https://github.com/lukemelas/EfficientNet-PyTorch to /tmp/pip-req-build-ivuw6xi5
  Running command git clone -q https://github.com/lukemelas/EfficientNet-PyTorch /tmp/pip-req-build-ivuw6xi5
  Created wheel for efficientnet-pytorch: filename=efficient

In [ ]:
#!pip install efficientnet-pytorch
#!pip install pretrainedmodels 

In [ ]:
!pip install git+https://github.com/qubvel/segmentation_models.pytorch

  Cloning https://github.com/qubvel/segmentation_models.pytorch to /tmp/pip-req-build-621hwehe
  Running command git clone -q https://github.com/qubvel/segmentation_models.pytorch /tmp/pip-req-build-621hwehe
  Using cached pretrainedmodels-0.7.4-py3-none-any.whl
  Using cached efficientnet_pytorch-0.6.3-py3-none-any.whl
  Created wheel for segmentation-models-pytorch: filename=segmentation_models_pytorch-0.2.0-py3-none-any.whl size=88601 sha256=4d5364bbb208898c0730d0d5ad052fac50d7a2eb1a6573e15da368fc592f94c3
  Stored in directory: /tmp/pip-ephem-wheel-cache-_fi2sojx/wheels/fa/c5/a8/1e8af6cb04a0974db8a4a156ebd2fdd1d99ad2558d3fce49d4
Successfully built segmentation-models-pytorch
  Attempting uninstall: pretrainedmodels
    Found existing installation: pretrainedmodels 0.7.0
    Uninstalling pretrainedmodels-0.7.0:
      Successfully uninstalled pretrainedmodels-0.7.0
  Attempting uninstall: efficientnet-pytorch
    Found existing installation: efficientnet-pytorch 0.7.1
    Uninstalling

# Restart Runtime now

In [ ]:
import torch
import numpy as np
import torch.nn as nn
import segmentation_models_pytorch as smp

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import numpy as np
import cv2
import matplotlib.pyplot as plt

In [ ]:
torch.cuda.is_available()

True

In [ ]:
DATA_DIR = '/content/drive/MyDrive/TheIRDataset'
#DATA_DIR = '/content/drive/MyDrive/psp-1.0'
# load repo with data if it is not exists
if not os.path.exists(DATA_DIR):
    print('Loading data...')
    os.system('git clone https://github.com/alexgkendall/SegNet-Tutorial ./data')
    print('Done!')

In [ ]:
x_train_dir = os.path.join(DATA_DIR, 'train')
y_train_dir = os.path.join(DATA_DIR, 'trainannot')

x_valid_dir = os.path.join(DATA_DIR, 'val')
y_valid_dir = os.path.join(DATA_DIR, 'valannot')

x_test_dir = os.path.join(DATA_DIR, 'test')
y_test_dir = os.path.join(DATA_DIR, 'testannot')


In [ ]:
# helper function for data visualization
def visualize(**images):
    """PLot images in one row."""
    n = len(images)
    plt.figure(figsize=(16, 5))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(' '.join(name.split('_')).title())
        plt.imshow(image)
    plt.show()

In [ ]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset as BaseDataset

In [ ]:
class Dataset(BaseDataset):
    """CamVid Dataset. Read images, apply augmentation and preprocessing transformations.
    
    Args:
        images_dir (str): path to images folder
        masks_dir (str): path to segmentation masks folder
        class_values (list): values of classes to extract from segmentation mask
        augmentation (albumentations.Compose): data transfromation pipeline 
            (e.g. flip, scale, etc.)
        preprocessing (albumentations.Compose): data preprocessing 
            (e.g. noralization, shape manipulation, etc.)
    
    """

    #0 - sky(d.blue). 1 - water(l.blue).   2 - bridge(yellow).   3 - obstacle(purple).  4- living ob(green).  
    #                5- backgnd (orange). 6 - self(pink)
    
    CLASSES = ['sky', 'water', 'bridge', 'obstacle', 'living obstacle', 'background', 'self']
                  
    
    def __init__(
            self, 
            images_dir, 
            masks_dir, 
            classes=None, 
            augmentation=None, 
            preprocessing=None,
    ):
        self.ids = os.listdir(images_dir)
        self.images_fps = [os.path.join(images_dir, image_id) for image_id in self.ids]
        self.masks_fps = [os.path.join(masks_dir, image_id) for image_id in self.ids]
        
        #mask1 = cv2.imread(self.masks_fps[1], cv2.IMREAD_UNCHANGED)
        #print("true value of image", np.unique(mask1))
        # convert str names to class values on masks
        self.class_values = [self.CLASSES.index(cls.lower()) for cls in classes]
        
        self.augmentation = augmentation
        self.preprocessing = preprocessing
    
    def __getitem__(self, i):
        
        # read data
        image = cv2.imread(self.images_fps[i])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        #HAD TO CHANGE FLAG 0 TO IM_READ_UNCHANGED BELOW as it was not reading the pixel values otherwise 
        # mask = cv2.imread(self.masks_fps[i], 0)
        mask = cv2.imread(self.masks_fps[i], cv2.IMREAD_UNCHANGED)
        #print ("masked image values", mask)
        #print ("masked shape values", mask.shape)
        #print("masked unique values", np.unique(mask))
        #print("pixel values of mask", mask)
        
        # extract certain classes from mask (e.g. cars)
        masks = [(mask == v) for v in self.class_values]
        #print ("masks extraction", masks)
        mask = np.stack(masks, axis=-1).astype('float')
        #print("stacks of classes for mask", masks)
        
        # apply augmentations
        if self.augmentation:
            sample = self.augmentation(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']
        
        # apply preprocessing
        if self.preprocessing:
            sample = self.preprocessing(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']
            #print('masked array first', np.unique(mask))
            
        return image, mask
        
    def __len__(self):
        return len(self.ids)

In [ ]:
# Lets look at data we have. it can only put 1 class at a time 

dataset = Dataset(x_train_dir, y_train_dir, classes=['background'])

image, mask = dataset[2569] # get some sample. 


'''
#just picking 1 image from the train dataset. Also selcting only class 'obstacle'. 
#the stacks of classes change when i change the class to sky. so the visualize is only showing the array for that specific class. 
# it's bollean - yes or No. Sp it will say if every pixel belongs to that class or not. it's able to print the ground truth properly
which means dataset is created properly. 
'''
#print ('masked array' , np.unique(mask))
visualize(
    image=image, 
    cars_mask=mask.squeeze(),
)


In [ ]:
import albumentations as albu

In [ ]:
def get_training_augmentation():
    train_transform = [

        albu.HorizontalFlip(p=0.5),

        albu.ShiftScaleRotate(scale_limit=0.5, rotate_limit=0, shift_limit=0.1, p=1, border_mode=0),

        albu.PadIfNeeded(min_height=320, min_width=320, always_apply=True, border_mode=0),
        albu.RandomCrop(height=320, width=320, always_apply=True),

        albu.IAAAdditiveGaussianNoise(p=0.2),
        albu.IAAPerspective(p=0.5),

        albu.OneOf(
            [
                albu.CLAHE(p=1),
                albu.RandomBrightness(p=1),
                albu.RandomGamma(p=1),
            ],
            p=0.9,
        ),

        albu.OneOf(
            [
                albu.IAASharpen(p=1),
                albu.Blur(blur_limit=3, p=1),
                albu.MotionBlur(blur_limit=3, p=1),
            ],
            p=0.9,
        ),

        albu.OneOf(
            [
                albu.RandomContrast(p=1),
                albu.HueSaturationValue(p=1),
            ],
            p=0.9,
        ),
    ]
    return albu.Compose(train_transform)


def get_validation_augmentation():
    """Add paddings to make image shape divisible by 32"""
    test_transform = [
        albu.PadIfNeeded(384, 480)
    ]
    return albu.Compose(test_transform)


'''
OpenCV img = cv2.imread(path) loads an image with HWC-layout (height, width, channels), 
while Pytorch requires CHW-layout. So we have to do np.transpose(image,(2,0,1))
for HWC->CHW transformation.
'''
def to_tensor(x, **kwargs):
    return x.transpose(2, 0, 1).astype('float32')


def get_preprocessing(preprocessing_fn):
    """Construct preprocessing transform
    
    Args:
        preprocessing_fn (callbale): data normalization function 
            (can be specific for each pretrained neural network)
    Return:
        transform: albumentations.Compose
    
    """
    
    _transform = [
        albu.Lambda(image=preprocessing_fn),
        albu.Lambda(image=to_tensor, mask=to_tensor),
    ]
    return albu.Compose(_transform)

In [ ]:
#### Visualize resulted augmented images and masks - thows error after 1st picture. 
#THIS IS NOT NECESSARY. plus our images have some augmentation
'''
augmented_dataset = Dataset(
    x_train_dir, 
    y_train_dir, 
    augmentation=get_training_augmentation(), 
    classes=['obstacle'],
)

# same image with different random transforms
#for i in range(3):  (Had to comment this out as i only have 1 image. will remove during actual training)
for i in range(3):
   image, mask = augmented_dataset[1]
   visualize(image=image, mask=mask.squeeze(-1))
   '''

In [ ]:
ENCODER = 'se_resnext50_32x4d'
ENCODER_WEIGHTS = 'imagenet'
CLASSES = ['sky', 'water', 'bridge', 'obstacle', 'living obstacle', 'background', 'self']
ACTIVATION = 'softmax2d' # could be None for logits or 'softmax2d' for multiclass segmentation
DEVICE = 'cuda'

# create segmentation model with pretrained encoder
model = smp.FPN(
    encoder_name=ENCODER, 
    #encoder_weights=ENCODER_WEIGHTS, Do not want to use existing encoder weights. 
    encoder_weights=None, 
    classes=len(CLASSES), 
    activation=ACTIVATION,
)

preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER, ENCODER_WEIGHTS)
'''if i use existing encoder weights, then it doesn't display all classes in the prediction for some reason. 
    have not found out exactly why. but perhaps std classes as sky / water are being taken up from the weights. 
    obstacle , backngd are not standard and therefore may not be plotted. So i will train the model from scratch
''' 


"if i use existing encoder weights, then it doesn't display all classes in the prediction for some reason. \n    have not found out exactly why. but perhaps std classes as sky / water are being taken up from the weights. \n    obstacle , backngd are not standard and therefore may not be plotted. So i will train the model from scratch\n"

In [ ]:
train_dataset = Dataset(
    x_train_dir, 
    y_train_dir, 
    #augmentation=get_training_augmentation(), try w/o augmentation 
    preprocessing=get_preprocessing(preprocessing_fn),
    classes=CLASSES,
)

valid_dataset = Dataset(
    x_valid_dir, 
    y_valid_dir, 
    #augmentation=get_validation_augmentation(), try w/o augmentation
    preprocessing=get_preprocessing(preprocessing_fn),
    classes=CLASSES,
)

train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True, num_workers=4)
#train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True, num_workers=12) - used for running iwth 1 image
valid_loader = DataLoader(valid_dataset, batch_size=5, shuffle=False, num_workers=1)

KeyboardInterrupt: ignored

In [ ]:

loss = smp.utils.losses.DiceLoss()
metrics = [
    smp.utils.metrics.IoU(threshold=0.5),
]

optimizer = torch.optim.Adam([ 
    dict(params=model.parameters(), lr=0.0001),
])

In [ ]:
train_epoch = smp.utils.train.TrainEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    optimizer=optimizer,
    device=DEVICE,
    verbose=True,
)

valid_epoch = smp.utils.train.ValidEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    device=DEVICE,
    verbose=True,
)

In [ ]:
###### THIS IS WHERE YOU ARE TRAINING THE MODEL. ONLY RUN IF YOU WANT TO TRAIN
max_score = 0

for i in range(0, 100):
    
    print('\nEpoch: {}'.format(i))
    train_logs = train_epoch.run(train_loader)
    valid_logs = valid_epoch.run(valid_loader)
    
    # do something (save model, change lr, etc.)
    if max_score < valid_logs['iou_score']:
        max_score = valid_logs['iou_score']
        torch.save(model, '/content/drive/MyDrive/Models/pspnet/pspnet-pytorch/best_model.pth')
        print('Model saved!')
        
    if i == 25:
        optimizer.param_groups[0]['lr'] = 1e-5
        print('Decrease decoder learning rate to 1e-5!')


Epoch: 0
train:   0%|          | 0/257 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


valid: 100%|██████████| 148/148 [04:25<00:00,  1.79s/it, dice_loss - 0.0755, iou_score - 0.8636]
Model saved!

Epoch: 1
valid: 100%|██████████| 148/148 [00:12<00:00, 11.72it/s, dice_loss - 0.07363, iou_score - 0.8664]
Model saved!

Epoch: 2
valid: 100%|██████████| 148/148 [00:12<00:00, 11.75it/s, dice_loss - 0.06658, iou_score - 0.8783]
Model saved!

Epoch: 3
valid: 100%|██████████| 148/148 [00:12<00:00, 11.54it/s, dice_loss - 0.06184, iou_score - 0.8867]
Model saved!

Epoch: 4
valid: 100%|██████████| 148/148 [00:12<00:00, 11.79it/s, dice_loss - 0.05906, iou_score - 0.8914]
Model saved!

Epoch: 5
valid: 100%|██████████| 148/148 [00:12<00:00, 11.73it/s, dice_loss - 0.0589, iou_score - 0.8916]
Model saved!

Epoch: 6
valid: 100%|██████████| 148/148 [00:12<00:00, 11.49it/s, dice_loss - 0.04939, iou_score - 0.9086]
Model saved!

Epoch: 7
valid: 100%|██████████| 148/148 [00:12<00:00, 11.68it/s, dice_loss - 0.049, iou_score - 0.9094]
Model saved!

Epoch: 8
valid: 100%|██████████| 148/148 [00:

In [ ]:
### THIS IS WHERE YOU LOAD THE MODEL FOR YOUR USE. FOR INFERENCE YOU CAN JUST START WITH THIS W/O TRAINING
best_model = torch.load('/content/drive/MyDrive/Models/pspnet/pspnet-pytorch/best_model.pth')

In [ ]:
# create test dataset
test_dataset = Dataset(
    x_test_dir, 
    y_test_dir, 
    #x_train_dir, - check with trained images to see if that gives any prediction
    #y_train_dir,
    #augmentation=get_validation_augmentation(), 
    preprocessing=get_preprocessing(preprocessing_fn),
    classes=CLASSES,
    
)

test_dataloader = DataLoader(test_dataset)

In [ ]:
## evaluate model on test set. THIS IS WHERE YOU ARE TESTING THE TEST DATA SET ON THE MODEL
test_epoch = smp.utils.train.ValidEpoch(
    model=best_model,
    loss=loss,
    metrics=metrics,
    device=DEVICE,
)

logs = test_epoch.run(test_dataloader)

valid:   0%|          | 1/370 [00:03<19:47,  3.22s/it, dice_loss - 0.01356, iou_score - 0.9734]

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


valid: 100%|██████████| 370/370 [03:58<00:00,  1.55it/s, dice_loss - 0.02212, iou_score - 0.9586]


In [ ]:
# test dataset without transformations for image visualization
test_dataset_vis = Dataset(
    x_test_dir, y_test_dir, 
    #x_train_dir, y_train_dir, - checking with train dataset 
    classes=CLASSES,
    
)

In [ ]:
from torchvision.utils import save_image
import torch
import torchvision

In [1]:
#CLASSES = ['sky', 'water', 'bridge', 'obstacle', 'living obstacle', 'background', 'self']
import numpy as np 
DEVICE = 'cuda'
for i in range(1):
    n = np.random.choice(len(test_dataset))
    print("Name of the image being predicted",os.listdir(x_test_dir)[n])
    image_vis = test_dataset_vis[n][0].astype('uint8')
    image, gt_mask = test_dataset[n]
        
    #image_vis = test_dataset_vis[10][0].astype('uint8')
    #image, gt_mask = test_dataset[10]
    
    #print('gt mask before squeeze', np.unique(gt_mask))
    
    gt_mask = gt_mask.squeeze()
    x_tensor = torch.from_numpy(image).to(DEVICE).unsqueeze(0)
    #x_tensor = torch.from_numpy(image).unsqueeze(0)
    pr_mask = best_model.predict(x_tensor)
    pr_mask1 = (pr_mask.squeeze().cpu().numpy().round())
    print("type of pr_mask",type(pr_mask))
    #print(pr_mask[0])
    #print("x shape = {0}".format(x_tensor.shape))
    print ("shape of pr mask", pr_mask.shape)
    #print ("shape of gt mask", gt_mask.shape)
    #print ("shape of pr mask1", pr_mask1.shape)

    #print("unique values of pr mask", np.unique(pr_mask))
    #print("unique values of gt mask", np.unique(gt_mask))
    #print("unique values of pr mask", np.unique(pr_mask1))
    #print("unique values of pr mask1", np.unique(pr_mask1))

    #want to save the array as an image#
    img1 = pr_mask.squeeze()
    print ("shape of img1 after squeeze", img1.shape)
    print ("type of img1 after squeeze", type(img1))
    # we are only saving 1 class at a time. since we want obstacles, i have put 3 in the array below#
    save_image(img1[3], 'img1.png')
    
    #CLASSES = ['sky', 'water', 'bridge', 'obstacle', 'living obstacle', 'background', 'self']
    #              0.      1.       2.        3             4                 5           6
    visualize(
        image=image_vis, 
        ground_truth_mask=gt_mask[3], 
        predicted_mask=pr_mask1[3]
    )


NameError: ignored

In [ ]:
'''
mask_image = cv2.imread("/content/img1.png", cv2.IMREAD_UNCHANGED)
print(np.unique(mask_image))
print(mask_image)
'''